In [140]:
import seaborn as sns
import pandas as pd
import numpy as np
sns.set()

df = pd.read_csv('../outputs/db.csv', index_col=0)

df_2017 = df.loc[((df['Year'] == 2017) & (df['Subject'] == 'All')), ['GDP per capita (World Bank)', 'GDP per capita (PWT)', 'Documents', 'H index', 'Citations', 'Self-citations', 'Population', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2017_economic = df.loc[((df['Year'] == 2017) & (df['Subject'] == 'All')), ['Country','GDP per capita (World Bank)', 'GDP per capita (PWT)']]

df_2007_science = df.loc[((df['Year'] == 2007) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2002_science = df.loc[((df['Year'] == 2002) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_1996_science = df.loc[((df['Year'] == 1996) & (df['Subject'] == 'All')), ['Country','Documents', 'H index', 'Citations', 'Self-citations', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_2007_2017 = pd.merge(df_2017_economic, df_2007_science, on="Country")

df_2002_2017 = pd.merge(df_2017_economic, df_2002_science, on="Country")

df_1996_2017 = pd.merge(df_2017_economic, df_1996_science, on="Country")


for i, column in enumerate(df.columns.values):
    print(f"{i} - {column}")

0 - Independent
1 - Sink OFC
2 - Conduit OFC
3 - Code
4 - Country
5 - Year
6 - Population
7 - Population 25 Years Before
8 - Population 25+
9 - Documents
10 - Citable documents
11 - Citations
12 - Self-citations
13 - Citations per Document
14 - H index
15 - Subject
16 - Avg. Adult Schooling
17 - GDP per capita (World Bank)
18 - GDP per capita (PWT)
19 - MAX TAMS
20 - REAL TAMS
21 - Documents / M-TAMS
22 - Documents / R-TAMS
23 - Citations / M-TAMS
24 - Citations / R-TAMS
25 - Gov. Effectiveness
26 - Charges for the use of intellectual property, payments (BoP, current US$)
27 - Charges for the use of intellectual property, receipts (BoP, current US$)
28 - Firms that spend on R&D (% of firms)
29 - High-technology exports (current US$)
30 - Industrial design applications, nonresident, by count
31 - Industrial design applications, resident, by count
32 - Labor force with advanced education (% of total working-age population with advanced education)
33 - Patent applications, nonresidents
34

In [141]:
df_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Population,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.474846,0.084728,0.045100,0.093015,0.059740,0.001700,0.014629,0.099587,0.006788,0.010944,0.087782,0.113089,0.117469,0.138244,0.162500
GDP per capita (PWT),0.474846,1.000000,0.057212,-0.036829,0.067929,0.051720,0.017238,-0.018795,0.051800,0.004790,0.010262,0.158790,0.067229,0.213769,0.139347,0.096971
Documents,0.084728,0.057212,1.000000,0.449806,0.996386,0.972996,0.560059,-0.049614,0.314533,0.752522,0.766858,0.241769,0.290925,0.156086,0.160300,0.346915
H index,0.045100,-0.036829,0.449806,1.000000,0.448831,0.371329,0.286655,-0.084809,0.431080,0.361762,0.361134,0.449453,0.521422,0.351491,0.341507,0.497661
Citations,0.093015,0.067929,0.996386,0.448831,1.000000,0.966363,0.508902,-0.045227,0.328698,0.707494,0.723307,0.273078,0.324131,0.186946,0.189648,0.364027
Self-citations,0.059740,0.051720,0.972996,0.371329,0.966363,1.000000,0.595342,-0.036484,0.235034,0.788041,0.803750,0.160580,0.195083,0.098210,0.102112,0.262366
Population,0.001700,0.017238,0.560059,0.286655,0.508902,0.595342,1.000000,-0.055847,-0.012681,0.947419,0.941839,-0.036167,-0.026526,-0.061982,-0.058332,0.057839
Citations per Document,0.014629,-0.018795,-0.049614,-0.084809,-0.045227,-0.036484,-0.055847,1.000000,-0.078414,-0.072372,-0.072251,-0.041167,-0.032014,0.311297,0.515009,0.007616
Avg. Adult Schooling,0.099587,0.051800,0.314533,0.431080,0.328698,0.235034,-0.012681,-0.078414,1.000000,0.117722,0.127574,0.562886,0.620981,0.454609,0.436526,0.761015
MAX TAMS,0.006788,0.004790,0.752522,0.361762,0.707494,0.788041,0.947419,-0.072372,0.117722,1.000000,0.999339,0.027361,0.050563,-0.017416,-0.011055,0.174303


In [142]:
df_2007_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.470948,0.190684,0.077984,0.200805,0.186811,0.097226,0.146171,0.096048,0.100855,0.150603,0.170513,0.167734,0.184984,0.187792
GDP per capita (PWT),0.470948,1.000000,0.147427,-0.002667,0.160142,0.158282,-0.019986,0.093923,0.071030,0.076091,0.152601,0.115918,0.150003,0.125659,0.146736
Documents,0.190684,0.147427,1.000000,0.402805,0.995421,0.976307,0.220940,0.436395,0.801607,0.807727,0.399648,0.432026,0.405159,0.428085,0.442787
H index,0.077984,-0.002667,0.402805,1.000000,0.384015,0.335699,0.180646,0.390749,0.372079,0.367720,0.431866,0.452765,0.425680,0.437874,0.428707
Citations,0.200805,0.160142,0.995421,0.384015,1.000000,0.983727,0.228464,0.430538,0.750485,0.757288,0.396578,0.427857,0.406218,0.428545,0.433871
Self-citations,0.186811,0.158282,0.976307,0.335699,0.983727,1.000000,0.190078,0.354694,0.755315,0.765284,0.295253,0.315108,0.309124,0.322617,0.346054
Citations per Document,0.097226,-0.019986,0.220940,0.180646,0.228464,0.190078,1.000000,0.102699,0.110295,0.112731,0.404064,0.422669,0.521348,0.538939,0.294321
Avg. Adult Schooling,0.146171,0.093923,0.436395,0.390749,0.430538,0.354694,0.102699,1.000000,0.279399,0.282277,0.664440,0.688582,0.613034,0.627513,0.778191
MAX TAMS,0.096048,0.071030,0.801607,0.372079,0.750485,0.755315,0.110295,0.279399,1.000000,0.999337,0.204000,0.226658,0.205455,0.220930,0.293643
REAL TAMS,0.100855,0.076091,0.807727,0.367720,0.757288,0.765284,0.112731,0.282277,0.999337,1.000000,0.209006,0.227494,0.208739,0.222207,0.300998


In [143]:
df_2002_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.482865,0.163742,0.092429,0.167377,0.156389,0.014749,0.142693,0.057330,0.061425,0.137305,0.157940,0.163544,0.180531,0.166873
GDP per capita (PWT),0.482865,1.000000,0.127734,0.025156,0.139352,0.137923,-0.058553,0.081560,0.043510,0.048932,0.137343,0.109268,0.147168,0.134925,0.121899
Documents,0.163742,0.127734,1.000000,0.550542,0.995321,0.976951,0.241952,0.403333,0.735451,0.740350,0.391340,0.412888,0.439340,0.450471,0.396131
H index,0.092429,0.025156,0.550542,1.000000,0.520767,0.451218,0.222054,0.472936,0.462507,0.455917,0.571784,0.603676,0.582010,0.601077,0.531697
Citations,0.167377,0.139352,0.995321,0.520767,1.000000,0.989656,0.244245,0.382730,0.695036,0.701489,0.371411,0.390175,0.426285,0.435796,0.371590
Self-citations,0.156389,0.137923,0.976951,0.451218,0.989656,1.000000,0.215823,0.327874,0.689089,0.698029,0.292456,0.303453,0.351177,0.355210,0.299263
Citations per Document,0.014749,-0.058553,0.241952,0.222054,0.244245,0.215823,1.000000,0.253708,0.151632,0.152549,0.474963,0.496756,0.543146,0.556059,0.403825
Avg. Adult Schooling,0.142693,0.081560,0.403333,0.472936,0.382730,0.327874,0.253708,1.000000,0.256610,0.258884,0.613937,0.641313,0.585101,0.600310,0.723692
MAX TAMS,0.057330,0.043510,0.735451,0.462507,0.695036,0.689089,0.151632,0.256610,1.000000,0.999320,0.196836,0.211608,0.227608,0.234565,0.240662
REAL TAMS,0.061425,0.048932,0.740350,0.455917,0.701489,0.698029,0.152549,0.258884,0.999320,1.000000,0.197094,0.211311,0.228023,0.235601,0.245678


In [144]:
df_1996_2017.corr()

,GDP per capita (World Bank),GDP per capita (PWT),Documents,H index,Citations,Self-citations,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
GDP per capita (World Bank),1.000000,0.462423,0.178547,0.105676,0.178409,0.166371,0.032982,0.148418,0.084852,0.079392,0.183543,0.173338,0.197379,0.191252,0.178137
GDP per capita (PWT),0.462423,1.000000,0.123942,0.011203,0.133426,0.133967,-0.005264,0.059484,0.067194,0.066481,0.161885,0.115415,0.160170,0.138638,0.128905
Documents,0.178547,0.123942,1.000000,0.500235,0.996205,0.972425,0.273937,0.472573,0.784079,0.778684,0.436234,0.471824,0.496678,0.509658,0.402977
H index,0.105676,0.011203,0.500235,1.000000,0.475493,0.431291,0.210422,0.452327,0.450964,0.447045,0.524851,0.541161,0.536292,0.531107,0.474170
Citations,0.178409,0.133426,0.996205,0.475493,1.000000,0.986511,0.271895,0.449232,0.760574,0.757058,0.411605,0.441868,0.477575,0.486978,0.376674
Self-citations,0.166371,0.133967,0.972425,0.431291,0.986511,1.000000,0.237161,0.384120,0.750011,0.751308,0.322706,0.338489,0.391789,0.391476,0.294999
Citations per Document,0.032982,-0.005264,0.273937,0.210422,0.271895,0.237161,1.000000,0.355236,0.226679,0.225372,0.549961,0.609588,0.650077,0.676556,0.474082
Avg. Adult Schooling,0.148418,0.059484,0.472573,0.452327,0.449232,0.384120,0.355236,1.000000,0.332971,0.328157,0.626867,0.664811,0.613340,0.630393,0.740708
MAX TAMS,0.084852,0.067194,0.784079,0.450964,0.760574,0.750011,0.226679,0.332971,1.000000,0.999132,0.256012,0.284587,0.304096,0.314405,0.266238
REAL TAMS,0.079392,0.066481,0.778684,0.447045,0.757058,0.751308,0.225372,0.328157,0.999132,1.000000,0.247593,0.271440,0.294608,0.302678,0.259978


In [145]:
df_96_17 = df.loc[((df['Year'] <= 2017) & (df['Sink OFC'] == "No")), ['Documents', 'H index', 'Citations', 'Self-citations', 'Population', 'Citations per Document', 'Avg. Adult Schooling', 'MAX TAMS', 'REAL TAMS', 'GDP per capita (World Bank)', 'GDP per capita (PWT)', 'Documents / M-TAMS', 'Documents / R-TAMS', 'Citations / M-TAMS', 'Citations / R-TAMS', 'Gov. Effectiveness']]

df_96_17.corr()

,Documents,H index,Citations,Self-citations,Population,Citations per Document,Avg. Adult Schooling,MAX TAMS,REAL TAMS,GDP per capita (World Bank),GDP per capita (PWT),Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
Documents,1.000000,0.399497,0.882640,0.864745,0.180024,0.007672,0.130030,0.285672,0.289910,0.014126,0.015909,0.378906,0.410513,0.337917,0.349875,0.143314
H index,0.399497,1.000000,0.381958,0.312803,0.245100,0.132471,0.396927,0.356721,0.359138,-0.000999,-0.014183,0.461603,0.494878,0.475316,0.487352,0.525208
Citations,0.882640,0.381958,1.000000,0.980410,0.119217,0.034442,0.124915,0.217327,0.220973,0.018242,0.018860,0.338633,0.369262,0.378996,0.394042,0.144182
Self-citations,0.864745,0.312803,0.980410,1.000000,0.132694,0.026337,0.097124,0.230430,0.234720,0.018689,0.021110,0.241475,0.262999,0.276340,0.286520,0.105589
Population,0.180024,0.245100,0.119217,0.132694,1.000000,-0.016408,-0.059516,0.925899,0.922502,0.008089,0.014148,-0.021370,-0.019369,-0.022194,-0.020710,0.007803
Citations per Document,0.007672,0.132471,0.034442,0.026337,-0.016408,1.000000,0.037520,-0.004668,-0.003266,0.015599,0.003444,0.021133,0.024945,0.099634,0.106704,0.145443
Avg. Adult Schooling,0.130030,0.396927,0.124915,0.097124,-0.059516,0.037520,1.000000,0.092736,0.092082,0.005104,0.005065,0.216238,0.227086,0.198029,0.198835,0.689853
MAX TAMS,0.285672,0.356721,0.217327,0.230430,0.925899,-0.004668,0.092736,1.000000,0.999299,0.009393,0.024272,0.012830,0.019014,0.010289,0.013744,0.110648
REAL TAMS,0.289910,0.359138,0.220973,0.234720,0.922502,-0.003266,0.092082,0.999299,1.000000,0.010390,0.025269,0.016756,0.021278,0.013212,0.015631,0.116333
GDP per capita (World Bank),0.014126,-0.000999,0.018242,0.018689,0.008089,0.015599,0.005104,0.009393,0.010390,1.000000,0.505855,0.015079,0.010855,0.022034,0.018465,0.061436


In [146]:
df

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
8,Yes,No,No,ALB,Albania,1996,3098.699,2202.186,1553.434,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2094.0
12,Yes,No,No,DZA,Algeria,1996,29266.415,14872.253,11543.461,34,...,NaN,150.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,4214.0
16,No,No,No,ASM,American Samoa,1996,54.209,27.982,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,1996,64.363,25.571,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2449.0
31,Yes,No,No,AZE,Azerbaijan,1996,7855.558,5284.518,3903.462,7,...,NaN,17.0,165.0,0.23533,NaN,NaN,NaN,NaN,NaN,1992.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2019,28515.829,21474.553,16060.699,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,Yes,No,No,VNM,Viet Nam,2019,96462.108,73651.220,60449.404,84,...,87.152702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,Yes,No,No,YEM,Yemen,2019,29161.922,14297.617,11664.767,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894,Yes,No,No,ZMB,Zambia,2019,17861.034,8869.745,6182.643,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
df.describe()

,Year,Population,Population 25 Years Before,Population 25+,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,...,Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
count,109701.000000,1.097010e+05,1.097010e+05,1.047290e+05,109701.000000,109701.000000,1.097010e+05,1.097010e+05,109701.000000,109701.000000,...,42122.000000,63874.000000,6.160400e+04,52310.000000,41255.000000,82977.000000,31469.000000,39049.000000,3.917400e+04,7.018100e+04
mean,2008.237190,4.103513e+04,2.879520e+04,2.398319e+04,1883.298675,1755.485976,3.889542e+04,1.232935e+04,20.250966,95.963665,...,79.302353,7165.435013,1.371149e+04,0.999237,2092.975751,12943.468964,574.405128,14871.311788,5.274063e+04,2.946723e+04
std,6.885539,1.445041e+05,1.041902e+05,8.643938e+04,16311.032302,14590.417062,4.096600e+05,1.743147e+05,34.323828,142.741815,...,6.429026,28619.654702,7.319617e+04,0.958078,1902.656232,47176.829981,645.634720,22593.701743,2.920477e+05,9.192842e+04
min,1996.000000,7.830000e-01,7.080000e-01,3.305700e+01,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,39.970699,1.000000,1.000000e+00,0.005440,5.911830,0.000000,0.125990,34.000000,1.000000e+00,1.000000e+00
25%,2002.000000,2.671361e+03,1.812158e+03,1.771362e+03,4.000000,4.000000,4.700000e+01,3.000000e+00,5.500000,20.000000,...,76.031502,55.000000,4.500000e+01,0.266370,438.603960,67.390000,88.812130,3880.000000,1.312000e+03,3.419000e+03
50%,2009.000000,9.023354e+03,5.935895e+03,5.034927e+03,29.000000,28.000000,3.870000e+02,4.200000e+01,13.670000,45.000000,...,80.005600,296.000000,2.670000e+02,0.645560,1513.007490,456.900000,355.686260,7605.000000,6.019000e+03,8.270000e+03
75%,2014.000000,2.845550e+04,1.711339e+04,1.453767e+04,309.000000,299.000000,4.290000e+03,6.830000e+02,25.500000,111.000000,...,83.488403,2875.000000,1.626000e+03,1.553460,3342.100850,5715.980000,732.629920,16030.000000,2.554100e+04,2.452200e+04
max,2019.000000,1.433784e+06,1.230020e+06,1.007108e+06,699412.000000,669877.000000,1.956422e+07,8.572143e+06,2889.000000,2386.000000,...,100.000000,313052.000000,1.393815e+06,4.952780,8065.887290,528263.250000,3766.861460,238246.000000,7.127191e+06,2.104411e+06


In [148]:
df.isna().sum()

Independent                                                                                           296
Sink OFC                                                                                              296
Conduit OFC                                                                                           296
Code                                                                                                  296
Country                                                                                                 0
Year                                                                                                    0
Population                                                                                              0
Population 25 Years Before                                                                              0
Population 25+                                                                                       4972
Documents                                     

In [149]:
df_saturated = df[(df['Subject'] == 'All')].iloc[:, 0:26].dropna()

df_saturated.corr()

,Year,Population,Population 25 Years Before,Population 25+,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,Avg. Adult Schooling,GDP per capita (World Bank),GDP per capita (PWT),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
Year,1.000000,0.013030,0.021641,0.023072,0.092801,0.086564,-0.015359,-0.012243,-0.310544,-0.032547,0.179085,-0.038876,-0.025524,0.044204,0.045723,0.194861,0.189866,-0.006360,-0.023005,-0.009791
Population,0.013030,1.000000,0.991306,0.988992,0.445631,0.469257,0.305790,0.333373,-0.070339,0.271426,-0.016307,0.016633,0.017719,0.919684,0.915197,-0.029370,-0.023204,-0.033228,-0.029141,0.027632
Population 25 Years Before,0.021641,0.991306,1.000000,0.999494,0.495797,0.521768,0.342242,0.369380,-0.068125,0.296858,0.023368,0.015681,0.018972,0.954620,0.950896,-0.008958,0.000855,-0.015225,-0.009130,0.055878
Population 25+,0.023072,0.988992,0.999494,1.000000,0.505658,0.531631,0.350212,0.378399,-0.066264,0.299489,0.026813,0.016745,0.020556,0.957789,0.955153,-0.002267,0.006377,-0.009994,-0.004834,0.062071
Documents,0.092801,0.445631,0.495797,0.505658,1.000000,0.998264,0.882532,0.870688,0.045555,0.442409,0.327397,0.042067,0.052095,0.702084,0.712633,0.292710,0.327950,0.257308,0.274467,0.346008
Citable documents,0.086564,0.469257,0.521768,0.531631,0.998264,1.000000,0.887589,0.877693,0.047888,0.453248,0.326373,0.040613,0.049423,0.725930,0.736147,0.288340,0.323353,0.257384,0.274644,0.347336
Citations,-0.015359,0.305790,0.342242,0.350212,0.882532,0.887589,1.000000,0.981941,0.149773,0.414416,0.309131,0.048273,0.056215,0.547571,0.556441,0.264226,0.298499,0.316355,0.337964,0.342272
Self-citations,-0.012243,0.333373,0.369380,0.378399,0.870688,0.877693,0.981941,1.000000,0.118579,0.342981,0.242855,0.048293,0.059722,0.570592,0.580985,0.180230,0.204106,0.224337,0.239441,0.255517
Citations per Document,-0.310544,-0.070339,-0.068125,-0.066264,0.045555,0.047888,0.149773,0.118579,1.000000,0.132335,-0.006438,0.008782,0.012482,-0.047965,-0.044953,0.110477,0.131529,0.350583,0.375677,0.180744
H index,-0.032547,0.271426,0.296858,0.299489,0.442409,0.453248,0.414416,0.342981,0.132335,1.000000,0.450596,0.007413,-0.019057,0.379009,0.379706,0.498468,0.543371,0.512932,0.533695,0.557979


In [150]:
df_saturated.drop(columns=['REAL TAMS', 'GDP per capita (PWT)', 'Documents / R-TAMS', 'Citations / R-TAMS'], inplace=True)
df_saturated



,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Documents,...,Self-citations,Citations per Document,H index,Subject,Avg. Adult Schooling,GDP per capita (World Bank),MAX TAMS,Documents / M-TAMS,Citations / M-TAMS,Gov. Effectiveness
8,Yes,No,No,ALB,Albania,1996,3098.699,2202.186,1553.434,69,...,116,12.81,72,All,8.2,46971.53,1.805793e+04,0.003821,0.048947,-0.688588
12,Yes,No,No,DZA,Algeria,1996,29266.415,14872.253,11543.461,370,...,783,14.44,178,All,5.0,95096.88,7.436126e+04,0.004976,0.071849,-1.088766
31,Yes,No,No,AZE,Azerbaijan,1996,7855.558,5284.518,3903.462,251,...,250,4.15,112,All,10.3,33291.35,5.443054e+04,0.004611,0.019137,-0.916390
32,Yes,No,No,ARG,Argentina,1996,35246.376,24259.564,18720.272,4247,...,18728,18.88,431,All,8.5,14485.51,2.062063e+05,0.020596,0.388850,0.165690
36,Yes,No,No,AUS,Australia,1996,18189.274,13033.130,11623.150,25536,...,153278,31.11,1001,All,11.8,31179.31,1.537909e+05,0.166044,5.165616,1.800565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,Yes,No,No,USA,United States of America,2017,325084.758,256990.608,219586.807,696160,...,2225802,7.05,11,All,13.4,54225.45,3.443674e+06,0.202156,1.425201,1.554298
854,Yes,No,No,BFA,Burkina Faso,2017,19193.236,9297.110,6683.564,560,...,462,5.09,111,All,1.5,17031.02,1.394567e+04,0.040156,0.204393,-0.579943
858,Yes,No,No,URY,Uruguay,2017,3436.645,3154.459,2209.904,1595,...,1215,6.37,193,All,8.7,20551.41,2.744379e+04,0.058119,0.370217,0.431783
860,Yes,No,No,UZB,Uzbekistan,2017,31959.774,21375.114,17145.037,615,...,359,4.31,94,All,11.5,62531.04,2.458138e+05,0.002502,0.010783,-0.557881


In [151]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt 
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import StandardScaler

hot_enc = OneHotEncoder(sparse=False)

df_test = df_saturated[['Year', 'GDP per capita (World Bank)', 'H index', 'Documents / M-TAMS', 'Gov. Effectiveness']]
# df_test = df_test.groupby(by=[df_saturated.index, 'Year']).agg({'GDP per capita (World Bank)': 'mean', 'Documents / M-TAMS': 'mean', 'H index': 'max', 'Gov. Effectiveness': 'mean' })

X = df_test.iloc[:, 2:5]
y = df_test.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)


lr = LinearRegression(normalize=True)

lr.fit(X_train,y_train)


LinearRegression(normalize=True)

In [152]:
lr.score(X_test, y_test)

0.0020487255662072767